<a href="https://colab.research.google.com/github/drdww/OPIM5641/blob/main/Module3/M3_1/CoveringModels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Linear Programming: Covering Models
**OPIM 5641: Business Decision Modeling - Dept. of Operations and Information Management - University of Connecticut**

----------------------------------

Related Readings:
* `Pyomo Cookbook`: https://github.com/jckantor/ND-Pyomo-Cookbook/blob/master/notebooks/02.01-Production-Models-with-Linear-Constraints.ipynb
* `Powell`: Chapter 9 (Linear Optimization)

## Background
The covering model calls for minimizing an objective (usually cost) subject to greater-than constraints on required coverage

In [ ]:
# import modules

%matplotlib inline
from pylab import *

import shutil
import sys
import os.path

if not shutil.which("pyomo"):
    !pip install -q pyomo
    assert(shutil.which("pyomo"))

if not (shutil.which("cbc") or os.path.isfile("cbc")):
    if "google.colab" in sys.modules:
        !apt-get install -y -qq coinor-cbc
    else:
        try:
            !conda install -c conda-forge coincbc 
        except:
            pass

assert(shutil.which("cbc") or os.path.isfile("cbc"))

from pyomo.environ import *

# Example: Trail Mix
*Section 9.3 (Powell) - Trail Mix*

**Problem Description:**
Dahlby Outﬁtters wishes to introduce packaged trail mix as a new product. The ingredients for the trail mix are seeds,raisins, ﬂakes, and two kinds of nuts. Each ingredient contains certain amounts of vitamins, minerals, protein,and calories. The marketing department has speciﬁed that the product be designed so that a certain minimum nutritional proﬁle is met. The decision problem is to determine the optimal product composition — that is, to minimize the product cost by choosing the amount for each of the ingredients in the mix. The following data summarizes the parameters of the problem:


The following data summarizes the parameters of the problem:

Component | Seeds | Raisins | Flakes | Pecans | Walnuts |Nutritional Requirement
--- | --- | --- | --- | --- | ---| ---
Vitamins | 10 | 20 | 10 | 30 | 20| 16
Minerals | 5 | 7 | 4 | 9 | 2 | 10
Protein | 1 | 4 | 10 | 2 | 1 | 15
Calories | 500 | 450 | 160 | 300 | 500 | 600
---
Cost/pound (USD) | 4 | 5 | 3 | 7 | 6

**Update (Oct 1, 2020):** this table used to say vitamins $\leq$ 20, it has now been updated to '16', which is correct in the constraints and code below.

**Define the Objective Function**

$Cost = 4S + 5R + 3F + 7P + 6W$

**Write the Constraints**

$Min(Z) = 4S + 5R + 3F + 7P + 6W$

subject to:
* $10S + 20R + 10F + 30P + 20W >= 16$ `vitamins`
* $5S + 7R + 4F + 9P + 2W >= 10$ `minerals`
* $1S + 4R + 10F + 2P + 1W >= 15$  `protein`
* $500S + 450R + 160F + 300P + 500W >= 600$ `calories`
* $S, R, F, P, W >= 0$ `non-negativity`


Great! Now that your problem is defined - go code it up and solve it.

In [ ]:
# declare the model
model = ConcreteModel()

# declare decision variables
model.s = Var(domain=NonNegativeReals) # s for seeds
model.r = Var(domain=NonNegativeReals) # r for raisin
model.f = Var(domain=NonNegativeReals) # flakes
model.p = Var(domain=NonNegativeReals) # pecans
model.w = Var(domain=NonNegativeReals) # walnuts

# declare objective
model.cost = Objective(
                      expr = 4*model.s + 5*model.r + 3*model.f + 7*model.p + 6*model.w, # values come from the table
                      sense = minimize)

# declare constraints
model.Constraint1 = Constraint(expr = 10*model.s + 20*model.r + 10*model.f + 30*model.p + 10*model.w >= 16) # vitamins
model.Constraint2 = Constraint(expr = 5*model.s + 7*model.r + 4*model.f + 9*model.p + 2*model.w >= 10) # minerals
model.Constraint3 = Constraint(expr = 1*model.s + 4*model.r + 10*model.f + 2*model.p + 1*model.w >= 15) # protein
model.Constraint4 = Constraint(expr = 500*model.s + 450*model.r + 160*model.f + 300*model.p + 500*model.w >= 600) # calories

In [ ]:
# show the model you've created
model.pprint()

5 Var Declarations
    f : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True : NonNegativeReals
    p : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True : NonNegativeReals
    r : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True : NonNegativeReals
    s : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True : NonNegativeReals
    w : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True : NonNegativeReals

1 Objective Declarations
    cost : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : 4*s + 5*r + 3*f + 7*p + 6*

In [ ]:
# solve it
SolverFactory('cbc', executable='/usr/bin/cbc').solve(model).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 7.535799523
  Upper bound: 7.535799523
  Number of objectives: 1
  Number of constraints: 5
  Number of variables: 6
  Number of nonzeros: 5
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: None
      Number of

In [ ]:
# show the results
print("Cost = ", model.cost(), ' USD')
print("Seeds = ", model.s(), ' pounds')
print("Raisins = ", model.r(), ' pounds')
print("Flakes = ", model.f(), ' pounds')
print("Pecans = ", model.p(), ' pounds')
print("Walnuts = ", model.w(), ' pounds')

Cost =  7.53579968  USD
Seeds =  0.47732697  pounds
Raisins =  0.33412888  pounds
Flakes =  1.3186158  pounds
Pecans =  0.0  pounds
Walnuts =  0.0  pounds


# Yuck! Add a lower bound and re-run

Yuck! That might meet nutritional standards, but it looks quite yucky (can you have trail mix without any nuts in it?!) We can add more constrains so that there's at least 0.1 pounds of each.

In [ ]:
# declare the model
model = ConcreteModel()

# declare decision variables
model.s = Var(domain=NonNegativeReals)
model.r = Var(domain=NonNegativeReals)
model.f = Var(domain=NonNegativeReals) 
model.p = Var(domain=NonNegativeReals)
model.w = Var(domain=NonNegativeReals) # bounds=(0.15,Inf))

# declare objective
model.cost = Objective(
                      expr = 4*model.s + 5*model.r + 3*model.f + 7*model.p + 6*model.w, # values come from the table
                      sense = minimize)

# declare constraints
model.Constraint1 = Constraint(expr = 10*model.s + 20*model.r + 10*model.f + 30*model.p + 10*model.w >= 16) # vitamins
model.Constraint2 = Constraint(expr = 5*model.s + 7*model.r + 4*model.f + 9*model.p + 2*model.w >= 10) # minerals
model.Constraint3 = Constraint(expr = 1*model.s + 4*model.r + 10*model.f + 2*model.p + 1*model.w >= 15) # protein
model.Constraint4 = Constraint(expr = 500*model.s + 450*model.r + 160*model.f + 300*model.p + 500*model.w >= 600) # calories
model.Constraint5 = Constraint(expr = 1*model.s >= 0.15) # some seeds (at least 0.15 pounds of seeds)
model.Constraint6 = Constraint(expr = 1*model.r >= 0.15) # some raisins
model.Constraint7 = Constraint(expr = 1*model.f >= 0.15) # some flakes
model.Constraint8 = Constraint(expr = 1*model.p >= 0.15) # some flakes
model.Constraint9 = Constraint(expr = 1*model.w >= 0.15) # some walnuts

# show the model you've created
model.pprint()

5 Var Declarations
    f : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  0.15 :  None :   inf : False :  True : NonNegativeReals
    p : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  0.15 :  None :   inf : False :  True : NonNegativeReals
    r : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  0.15 :  None :   inf : False :  True : NonNegativeReals
    s : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  0.15 :  None :   inf : False :  True : NonNegativeReals
    w : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  0.15 :  None :   inf : False :  True : NonNegativeReals

1 Objective Declarations
    cost : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : 4*s + 5*r + 3*f + 7*p + 6*

In [ ]:
# solve it
SolverFactory('cbc', executable='/usr/bin/cbc').solve(model).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 8.332128099
  Upper bound: 8.332128099
  Number of objectives: 1
  Number of constraints: 5
  Number of variables: 6
  Number of nonzeros: 5
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: None
      Number of

In [ ]:
# show the results
print("Cost = ", model.cost(), ' USD')
print("Seeds = ", model.s(), ' pounds')
print("Raisins = ", model.r(), ' pounds')
print("Flakes = ", model.f(), ' pounds')
print("Pecans = ", model.p(), ' pounds')
print("Walnuts = ", model.w(), ' pounds')

Cost =  8.332128  USD
Seeds =  0.3911157  pounds
Raisins =  0.15  pounds
Flakes =  1.3558884  pounds
Pecans =  0.15  pounds
Walnuts =  0.15  pounds


Much better! A little more expensive, but def more yummy. Tell marketing you have a product worth selling now!

In [ ]:
# can we show how the constraints were satisfied?
print("Vitamin Content: ", model.Constraint1())
print("Minerals Content: ",model.Constraint2())
print("Protein Content: ",model.Constraint3())
print("Calories Content: ",model.Constraint4())

Vitamin Content:  24.6420053
Minerals Content:  10.00000021
Protein Content:  15.00000049
Calories Content:  600.000009


# Analysis of Output
Dig a little deeper - show much of each ingredient you are using (proportion, not raw numbers).

In [ ]:
# do we need to show binding constraints here? easier way to do this?
print("Vitamin Constraint: ", model.Constraint1())
print("Minerals Constraint: ",model.Constraint2())
print("Protein Constraint: ",model.Constraint3())
print("Calories Constraint: ",model.Constraint4())
# print("Seeds Supply Constraint:", model.Constraint5(),' pounds')
# print("Raisins Supply Constraint:", model.Constraint6(),' pounds')
# print("Flakes Supply Constraint:", model.Constraint7(),' pounds')
# print("Pecans Supply Constraint:", model.Constraint8(),' pounds')
# print("Walnuts Supply Constraint:", model.Constraint9(),' pounds')

Vitamin Constraint:  24.6420053
Minerals Constraint:  10.00000021
Protein Constraint:  15.00000049
Calories Constraint:  600.000009


AttributeError: ignored

# On Your Own
Try to explore shadow prices for other ingredients! Make similar plots to the furniture example.